In [224]:
import pandas as pd
import requests
import folium
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans

import matplotlib.cm as cm
import matplotlib.colors as colors
import numpy as np

# Scraping Wikipedia

We read from the HTML table using pandas directly. This returns a list of dataframes, and we are interested in the first dataframe. We take only the rows of the dataframe where `'Borough'` is _not_ `'Not assigned'`. We reset the index numbering.

In [225]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df = pd.read_html(url, header=0)[0]
df = df[df['Borough'] != 'Not assigned']
df.reset_index(drop=True, inplace=True)
df.head()

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


At this point, the dataframe looks almost right, but the instructions imply that some postal codes may be listed twice. Let's check if any postal code appears more than once in the dataframe:

In [226]:
(pd.value_counts(df['Postal code']) >1).any()

False

Great, now we need to make sure the Neighborhoods are separated by commas instead of forward slashes.

In [227]:
df['Neighborhood'] = df['Neighborhood'].str.replace(' /', ',')
df.head()

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Now we need to check for assigned boroughs with "Not assigned" neighborhoods.

In [228]:
(df['Neighborhood'] == 'Not assigned').any()

False

There do not appear to be any. Apparently the dataset has changed since the assignment was written.

# Get Latitude and Longitude

Now we load and inspect the latitude and longitude data from the provided csv file.

In [229]:
!wget --quiet http://cocl.us/Geospatial_data -O lat_lng.json

In [230]:
lat_lng = pd.read_csv('lat_lng.json')
lat_lng.rename(columns={'Postal Code': 'Postal code'}, inplace=True)

In [231]:
lat_lng

,Postal code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


Merge the two dataframes.

In [232]:
df = pd.merge(df, lat_lng)
df

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,Business reply mail Processing CentrE,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


# Exploring and clustering neighborhods

Let's start by creating a map of the neighborhoods.

We use geopy to find the latitude and longitude of Toronto

In [233]:
geolocator = Nominatim(user_agent='toronto')
location = geolocator.geocode('Toronto, Canada')
latitude, longitude = location.latitude, location.longitude

latitude, longitude

(43.6534817, -79.3839347)

And we create a map using folium

In [234]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neigh in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{} / {}'.format(neigh, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='black',
        fill=True,
        fill_color='red',
        fill_opacity=0.2,
        parse_html=False).add_to(map_toronto)  
map_toronto

Now let's use Foursquare to explore the Neighborhoods. We define our credentials.

In [235]:
CLIENT_ID = 'ESQILQUJ45MEFP5CYAUTQAPRNT1YA22FASZDU0VCI5VIE2EP'
CLIENT_SECRET = 'MZIKSDQUL4OHRIPALRKSUQSKUMMVKM2PCGAMGJPWKJ3G0MBZ'
VERSION = '20180605' # Foursquare API version
LIMIT = 100

Define a function to get venues nearby each neighborhood.

In [236]:
def get_nearby(names, lats, lngs, radius=500):
    venues_list = []
    for name, lat, lng in zip(names, lats, lngs):
#         print(name)
        
        # Create API request
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [237]:
venues = get_nearby(names=df['Neighborhood'],
                    lats=df['Latitude'],
                    lngs=df['Longitude']
                                  ) # this may take ~30 seconds

In [238]:
venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


Let's use one-hot encoding for the venue categories and make the first column `'Neighborhood'`.

In [239]:
venues_onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")
venues_onehot.drop(columns=['Neighborhood'], inplace=True)
venues_onehot.insert(0, 'Neighborhood', venues['Neighborhood'])
venues_onehot.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Now group by neighborhood:

In [240]:
venues_grouped = venues_onehot.groupby('Neighborhood').mean().reset_index()
venues_grouped.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.035714,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Perform $k$-means to cluster the neighborhood for a range of $k$. Let's start with $k$=3.

In [241]:
k = 3
kmeans = KMeans(n_clusters=k, random_state=0) \
               .fit(venues_grouped.drop('Neighborhood', 1))

We will get the top N venues for each neighborhood and store them in the columns V1...VN.

In [242]:
N = 5

# make N new columns
for i in range(N):
    venues_grouped['V{}'.format(i+1)] = ''

In [243]:
venues_grouped.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,V1,V2,V3,V4,V5
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,,,,,
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,,,,,
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,,,,,
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,,,,,
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.035714,...,0.0,0.0,0.0,0.0,0.0,,,,,


In [244]:
venues_grouped.loc[0, 'Airport']#'Agincourt', 'V0']

0.0

In [245]:
for i in range(venues_grouped.shape[0]): # loop over rows
    v = venues_grouped.iloc[i, 1:-N] # get each row (skipping 'Neighborhood' column and new empty columns)
    v.sort_values(ascending=False, inplace=True) # sort in descending order
    for j in range(N): # get top N venues
        venues_grouped.loc[df.index[i], 'V{}'.format(j+1)] = v.index[j]

Extract a dataframe with just the top N venues for each neighborhood.

In [246]:
v_topN = venues_grouped[['Neighborhood', *['V{}'.format(j+1) for j in range(N)]]]
v_topN.head()

,Neighborhood,V1,V2,V3,V4,V5
0,Agincourt,Lounge,Skating Rink,Latin American Restaurant,Clothing Store,Breakfast Spot
1,"Alderwood, Long Branch",Pizza Place,Gym,Sandwich Place,Dance Studio,Pub
2,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Middle Eastern Restaurant,Pharmacy,Sushi Restaurant
3,Bayview Village,Café,Bank,Japanese Restaurant,Chinese Restaurant,Yoga Studio
4,"Bedford Park, Lawrence Manor East",Italian Restaurant,Thai Restaurant,Restaurant,Coffee Shop,Sandwich Place


Add the cluster label to the dataframe of the top N venues.

In [247]:
v_topN.insert(1, 'Cluster label', kmeans.labels_)
v_topN.head()

,Neighborhood,Cluster label,V1,V2,V3,V4,V5
0,Agincourt,0,Lounge,Skating Rink,Latin American Restaurant,Clothing Store,Breakfast Spot
1,"Alderwood, Long Branch",0,Pizza Place,Gym,Sandwich Place,Dance Studio,Pub
2,"Bathurst Manor, Wilson Heights, Downsview North",0,Bank,Coffee Shop,Middle Eastern Restaurant,Pharmacy,Sushi Restaurant
3,Bayview Village,0,Café,Bank,Japanese Restaurant,Chinese Restaurant,Yoga Studio
4,"Bedford Park, Lawrence Manor East",0,Italian Restaurant,Thai Restaurant,Restaurant,Coffee Shop,Sandwich Place


Now let's merge this with the dataframe with latitute and longitude information.

In [268]:
v_topN

,Neighborhood,Cluster label,V1,V2,V3,V4,V5
0,Agincourt,0,Lounge,Skating Rink,Latin American Restaurant,Clothing Store,Breakfast Spot
1,"Alderwood, Long Branch",0,Pizza Place,Gym,Sandwich Place,Dance Studio,Pub
2,"Bathurst Manor, Wilson Heights, Downsview North",0,Bank,Coffee Shop,Middle Eastern Restaurant,Pharmacy,Sushi Restaurant
3,Bayview Village,0,Café,Bank,Japanese Restaurant,Chinese Restaurant,Yoga Studio
4,"Bedford Park, Lawrence Manor East",0,Italian Restaurant,Thai Restaurant,Restaurant,Coffee Shop,Sandwich Place
...,...,...,...,...,...,...,...
91,"Willowdale, Newtonbrook",1,Home Service,Yoga Studio,Doner Restaurant,Dim Sum Restaurant,Diner
92,Woburn,0,Coffee Shop,Indian Restaurant,Korean Restaurant,Donut Shop,Dim Sum Restaurant
93,Woodbine Heights,0,Dance Studio,Video Store,Beer Store,Diner,Bus Stop
94,York Mills West,2,Park,Bank,Convenience Store,Empanada Restaurant,Electronics Store


In [298]:
merge = list(v_topN.merge(df, 'left', on=['Neighborhood'])['Neighborhood'])
orig = list(v_topN['Neighborhood'])

set(merge) <= set(orig), set(merge) >= set(orig)

(True, True)

In [289]:
len(orig), len(merge)

(96, 101)

In [256]:
df['Neighborhood'], venues_grouped['Neighborhood']

(0                                              Parkwoods
 1                                       Victoria Village
 2                              Regent Park, Harbourfront
 3                       Lawrence Manor, Lawrence Heights
 4            Queen's Park, Ontario Provincial Government
                              ...                        
 98         The Kingsway, Montgomery Road, Old Mill North
 99                                  Church and Wellesley
 100                Business reply mail Processing CentrE
 101    Old Mill South, King's Mill Park, Sunnylea, Hu...
 102    Mimico NW, The Queensway West, South of Bloor,...
 Name: Neighborhood, Length: 103, dtype: object,
 0                                           Agincourt
 1                              Alderwood, Long Branch
 2     Bathurst Manor, Wilson Heights, Downsview North
 3                                     Bayview Village
 4                   Bedford Park, Lawrence Manor East
                            ...       

In [223]:
venues.insert(1, 'Cluster label', kmeans.labels_)
venues.head()

ValueError: Length of values does not match length of index

We will add the cluster label as a color on the map

In [221]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set up color scheme
rainbow = [colors.rgb2hex(i) for i in cm.rainbow(np.linspace(0,1,k))]

for lat, lng, cluster, neigh in zip(venues['Latitude'], venues['Longitude'], venues['Cluster label'], venues['Neighborhood']):
    label = '{} / {}'.format(neigh, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color='red',
        fill_opacity=0.2,
        parse_html=False).add_to(map_clusters)  
map_clusters

KeyError: 'Latitude'